### TogetherAI Allow free Use of 11B Llama VLM but it does allow us to set instruction


1) Since the usage of ***Llama 11 B*** is freely available from ***Together AI*** i wanted to tes how it performs. The results are satisfactory and consistant only problem is that It can be finetuned further and also it can not take instructions such as provide us result in ***JSON etc***. for that another model such as ***Llama 11b Instruct*** is awesome but we need to think about hosting it.

In [ ]:
from together import Together
import base64


client = Together(api_key = '****')


image_path = r"C:\Users\Utente\Projects\ai-expense-buddy\data\yep.jpg"  

def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

base64_image = encode_image(image_path)

print(base64)       

query = """
        provide me DESCRIZIONE with PREZZO and IVA, DATA and Seller.
    """
messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": query},
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
                },
            ],
        }
    ]

response = client.chat.completions.create(
    model="meta-llama/Llama-Vision-Free",
    messages=messages,
    max_tokens=None,
    temperature=0.7,
    top_p=0.7,
    top_k=50,
    repetition_penalty=1,
    stop=["<|eot_id|>", "<|eom_id|>"],
    stream=True,
)


text = ''
for chunk in response:
    if len(chunk.choices) != 0:
        print(chunk.choices[0].delta.content or "", end="", flush=True)
        text += chunk.choices[0].delta.content + ' '


<module 'base64' from 'c:\\Users\\Utente\\miniconda3\\envs\\OCR_tests\\Lib\\base64.py'>
**Invoice Details**

*   **Description**: Kaleem Ullah 2
*   **Price**: 684,00
*   **VAT**: 22,00%
*   **Date**: 11-10-2024 13:01
*   **Seller**: Defanti's Club S.r.l.

### PaddleOCR

1) Another Method that I think could work is that we extract the text manually with the help of best OCR Extraction tool out there in this case I found PaddleOCR which works like charm. So I plan to use this to extract the text 

In [ ]:
from paddleocr import PaddleOCR

# Paddleocr supports Chinese, English, French, German, Korean and Japanese
# You can set the parameter `lang` as `ch`, `en`, `french`, `german`, `korean`, `japan`
# to switch the language model in order
ocr = PaddleOCR(use_angle_cls=True, lang='it') # need to run only once to download and load model into memory
img_path = r"C:\Users\Utente\Projects\ai-expense-buddy\data\WhatsApp Image 2024-11-03 at 17.07.02.jpeg"
result = ocr.ocr(img_path, cls=True)

text = ' '
for idx in range(len(result)):
    res = result[idx]
    for line in res:
        #print(line)
        text += line[1][0] + ' '

print(text)

[2024/11/18 22:37:39] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Utente/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Utente/.paddleocr/whl\\rec\\latin\\latin_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_ba

In [17]:

from langchain.embeddings import HuggingFaceEmbeddings      
from langchain.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from transformers import AutoTokenizer, AutoModelForCausalLM,  pipeline
from langchain.llms import HuggingFacePipeline

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/distilbert-multilingual-nli-stsb-quora-ranking")

# Load the LLM pipeline
llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0,
    max_length=3000,  # Increase max_length as per your input size
    max_new_tokens=1000  # Control the output length
)


llm = HuggingFacePipeline(pipeline=llm_pipeline)

documents = [
    text
]

# Create the FAISS index with embeddings
vector_store = FAISS.from_texts(documents, embedding_model)

# Step 3: Set up the retriever
retriever = vector_store.as_retriever()

system_prompt = (
    "You are an helpful and friendly assistant for question-answering tasks. "
    "The text you will be seeing is extracted from italian invoice"
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, Just say 'Text Doesnt Mention it'"
    "Answer in very pricise manner"
    "You need to provide me the following details in json format VenderName , TotalPrice , Dataofpurchase , Each Item withs price"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)




C:\Users\Utente\AppData\Local\Temp\ipykernel_11952\3466856374.py:25: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)


In [20]:


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": 'Extract me the useful information'})['answer']


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=1000) and `max_length`(=3000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [25]:
print(results.split('Extracted Information')[1])



- VenderName: MIT MD SpA
- TotalPrice: €33.80
- Dataofpurchase: 03/11/24
- Each Item withs price:
  - Coni di Mais: €4.00
  - Bio: €10.00
  - Subtot: €6.51
  - Totale di cui IVA: €6.51
  - Importo Pago: €6.51
  - Data: 03/11/24
  - Tipo di pagamento: EFT-POS
  - Forma di pagamento: UcMastercard
  - ID: A.ID
  - CVM: No CVM
  - CVM: No CVM
  - Tipo di pagamento: No CVM
  - Tipo di pagamento: No CVM
  - Tipo di pagamento: No CVM

The final answer is: 
```
{
  "VenderName": "MIT MD SpA",
  "TotalPrice": "€33.80",
  "Dataofpurchase": "03/11/24",
  "Each Item withs price": 
  {
    "Coni di Mais": "€4.00",
    "Bio": "€10.00",
    "Subtot": "€6.51",
    "Totale di cui IVA": "€6.51",
    "Importo Pago": "€6.51",
    "Data": "03/11/24",
    "Tipo di pagamento": "EFT-POS",
    "Forma di pagamento": "UcMastercard",
    "ID": "A.ID",
    "CVM": "No CVM",
    "CVM": "No CVM",
    "Tipo di pagamento": "No CVM",
    "Tipo di pagamento": "No CVM",
    "Tipo di pagamento": "No CVM"
  }
}
```
